# Model Selection

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy.stats import beta, betabinom, binom
from scipy.special import binom as binom_coef
from scipy.special import beta as beta_func
%matplotlib inline
import pandas as pd

from IPython.display import Image
from IPython.display import IFrame

### Problem 1: Binomial distribution

Let's say we're modelling $N$ iid trials with the Binomial distribution, i.e. the likelihood of the model is
$$p(k|N,q) = \mathcal{Bin}(k|N,q) = C_N^kq^k(1-q)^{N-k}.$$
For the parameter $q$, we consider the mixture of Beta distributions as the prior distribution, i.e.
$$p(q|w,a,b) = \sum_jw_j\mathcal{Beta}(q|a_j,b_j),\quad\mathcal{Beta}(q|a,b) = \frac{1}{\mathcal{B}(a,b)}q^{a-1}(1-q)^{b-1},\quad w_j,a_j,b_j>0,\quad\sum_jw_j=1.$$

The model selection is the selection of $w,a,b$. We're going to study these three following priors
1. "fair coin" model: $a=b=10$
2. "one-sided coin" model: $a=b=0.5$
3. "unfair coin" model: mixture of $a_1=15, b_1=3$ (with $w_1 = 0.5$) and $a_2=3, b_2=15$ (with $w_2 = 0.5$)

#### Exercise 1.1: priors (2 points)
Plot the pdfs of the priors. Does it match the meaning of their description above?

In [ ]:
def pdf_prior(q,w,a,b):
    #YOUR CODE HERE

In [ ]:
w_models = [np.array([1.0]), np.array([1.0]), np.array([0.5, 0.5])]
a_models = [np.array([10]), np.array([0.5]), np.array([15, 3])]
b_models = [np.array([10]), np.array([0.5]), np.array([3, 15])]

q = np.linspace(0,1,200)
for i in range(3):
    plt.plot(q,pdf_prior(q,w_models[i],a_models[i],b_models[i]), label=f'prior {i+1}')
plt.legend()
plt.xlabel('q')
plt.ylabel('prior: p(q|w,a,b)')
plt.grid()

#### Exercise 1.2: evidence (5 points)

Derive the formula for evidence, which distribution it corresponds to? Implement the function evaluating evidence and plot the evidence.
$$p(k|w,a,b) = \int dq\; p(k|q,N)p(q|w,a,b) = ?$$
Make conclusions in the end.

In [ ]:
def p_evidence(k,N,w,a,b):
    #YOUR CODE HERE

In [ ]:
N = 100
k = np.arange(0,N+1,1)
for i in range(3):
    plt.plot(k,p_evidence(k,N,w_models[i],a_models[i],b_models[i]), label=f'prior {i+1}')
plt.legend()
plt.xlabel('k')
plt.ylabel('evidence: p(k|w,a,b)')
plt.grid()

In [ ]:
N = 100
k = np.arange(0,N+1,1)
k_observed = 80
for i in range(3):
    plt.plot(k,p_evidence(k,N,w_models[i],a_models[i],b_models[i]), label=f'prior {i+1}')
    print('Evidence for model {}: {}'.format(i+1,p_evidence(k_observed,N,w_models[i],a_models[i],b_models[i])))
    plt.scatter(k_observed,p_evidence(k_observed,N,w_models[i],a_models[i],b_models[i]), marker='*')
plt.legend()
plt.xlabel('k')
plt.ylabel('evidence: p(k|w,a,b)')
plt.grid()

Conclusion: ...

#### Exercise 1.3: training (5 points)

In Bayesian inference, training of a model corresponds to finding the posterior distribution. For our problem, this is the distribution of $q$ for given, $w,a,b,N$ and observed data $k$, i.e.
$$p(q|k,w,a,b) \propto p(k|q)p(q|w,a,b) \propto ?$$
Remember, that the normalization constant of our posterior is the evidence. Indeed,
$$p(q|k,w,a,b) = \frac{p(k|q)p(q|w,a,b)}{\int dq\; p(k|q)p(q|w,a,b)}\,, \;\; p(k|w,a,b) = \int dq\, p(k|q)p(q|w,a,b)\,.$$

Derive the density of the posterior distribution, which distribution it corresponds to? Implement the formula for the pdf of the posterior, plot it, and compare to the pdfs of the prior.

In [ ]:
def pdf_posterior(q,k,N,w,a,b):
    #YOUR CODE HERE

In [ ]:
q = np.linspace(0,1,200)
colors = ['C0','C1','C2']
for i in range(3):
    plt.plot(q,pdf_prior(q,w_models[i],a_models[i],b_models[i]), '--', c=colors[i], label=f'prior {i+1}')
    plt.plot(q,pdf_posterior(q,k_observed,N,w_models[i],a_models[i],b_models[i]), c=colors[i], label=f'posterior {i+1}')
plt.legend()
plt.xlabel('q')
plt.ylabel('pdfs')
plt.title('priors vs posteriors')
plt.grid()

What happens if the dataset is smaller?

In [ ]:
k_small,N_small = 8,10

q = np.linspace(0,1,200)
colors = ['C0','C1','C2']
for i in range(3):
    plt.plot(q,pdf_prior(q,w_models[i],a_models[i],b_models[i]), '--', c=colors[i], label=f'prior {i+1}')
    plt.plot(q,pdf_posterior(q,k_small,N_small,w_models[i],a_models[i],b_models[i]), c=colors[i], label=f'posterior {i+1}')
plt.legend()
plt.xlabel('q')
plt.ylabel('pdfs')
plt.title('priors vs posteriors')
plt.grid()

#### Exercise 1.4: prediction (3 points)

For a single prior with parameters $w,a,b$, the predictive distribution is defined as
$$p(k_{new}|k,w,a,b) = \int dq\; p(k_{new}|q,N_{new})p(q|k,N,w,a,b) = ?$$

However, we can also define the predictive distribution as an ensemble over different priors weighted proportionally to their evidence, i.e.
$$p(k_{new}|k) = \sum_m p(k_{new},m|k) = \sum_m p(k_{new}|k,m)p(m|k) = \sum_m \bigg[\int dq\; p(k_{new}|q,N_{new})p(q|k,N,w,a,b)\bigg] p(m|k)\,,$$
where $p(m|k) \propto p(k|m=(w,a,b))p(m)$ and $p(m)$ is the uniform prior over models.

Derive the formula for the predictive distribution $p(k_{new}|k,w,a,b)$ and implement it. Implement the formula for the predictive distribution $p(k_{new}|k)$ using the implementation of the evidence evaluation and the formula of individual predictive distributions $p(k_{new}|k,w,a,b)$.

In [ ]:
def predictive_distribution_single(k_new,N_new,k,N,w,a,b):
    #YOUR CODE HERE

def predictive_distribution_all(k_new,N_new,k,N,w,a,b):
    #YOUR CODE HERE

In [ ]:
N_new = 100
k_new = np.arange(0,N_new+1,1)
for i in range(3):
    print('Evidence for model {}: {}'.format(i+1,p_evidence(k_small,N_small,w_models[i],a_models[i],b_models[i])))
    plt.plot(k_new,predictive_distribution_single(k_new,N_new,k_small,N_small,w_models[i],a_models[i],b_models[i]),
             '--',c=colors[i],label=f'model {i+1}')
plt.plot(k_new,predictive_distribution_all(k_new,N_new,k_small,N_small,w_models,a_models,b_models),
             c='C3',label = 'joint')
plt.legend()
plt.xlabel(r'$k_{new}$')
plt.ylabel(r'$p(k_{new}|k)$')
plt.title('Predictive distributions')
plt.grid()

### Problem 2. Game of Thrones

Let's apply our BetaMix-Binomial model for modelling the survival rate of characters in Game of Thrones using the dataset from https://www.kaggle.com/mylesoneill/game-of-thrones.

In [ ]:
data = pd.read_csv("character-predictions.csv")
data.head()

In [ ]:
data['People'] = 1
houses = data.groupby('house').sum()
houses_short = houses[houses['People']>=10].copy()
houses_short.head()

In [ ]:
p = houses_short['isAlive'].plot.barh(figsize = (8, 6),color='C0',label = "Alive")
(-houses_short['People']+houses_short['isAlive']).plot.barh(color='C1',label = "Dead")
p.legend()
p.axvline(0, color = 'k')
_ = p.set(xlabel = "Number of people")

In [ ]:
p = (houses_short['isAlive']/houses_short['People']).plot.barh(figsize = (8, 6),color='C0')
_ = p.set(xlabel = "Survival rate")

George Martin's modelling of different houses:
1. "fair coin" model: $a=b=10$. Martin is neutral, i.e., in every house, approximately half of characters survives.
2. "one-sided coin" model: $a=b=0.5$. Every house Martin either loves (almost everyone survives) or hates (almost none survives)
3. "unfair coin" model: mixture of $a_1=15, b_1=3$ and $a_2=3, b_2=15$. Every house Martin either likes (majority survives) or dislikes (minority survives)

In [ ]:
w_models = [np.array([1.0]), np.array([1.0]), np.array([0.5, 0.5])]
a_models = [np.array([10]), np.array([0.5]), np.array([12, 5])]
b_models = [np.array([10]), np.array([0.5]), np.array([5, 12])]

q = np.linspace(0,1,200)
for i in range(3):
    plt.plot(q,pdf_prior(q,w_models[i],a_models[i],b_models[i]), label=f'prior {i+1}')
plt.legend()
plt.xlabel('q')
plt.ylabel('prior: p(q|w,a,b)')
plt.grid()

#### Exercise 2.1: modelling houses (2 points)

For every single house, let's evaluate the evidence under different priors. For the number of success trials, we take the number of characters that survived. For the total number we take the number of characters in the house.

In [ ]:
houses_short['ev_1'],houses_short['ev_2'],houses_short['ev_3'] = 0.,0.,0.

for i in range(3):
    #YOUR CODE HERE        

In [ ]:
p = (houses_short[['ev_1','ev_2','ev_3']]).plot.barh(figsize = (8, 8))
p.set(xlabel = "Evidence")

We see that for different houses we should use different priors...

### What's in Martin's head?
Consider three different models of Martin's head.
1. One prior and one $q$ for all the houses. 
$$k_i \sim p(k_i|q)\,,\;\; q \sim p(q|w,a,b)$$
2. Individual $q$ and individual prior for every house.
$$k_i \sim p(k_i|q_i)\,,\;\; q_i \sim p(q_i|m_i=(w_i,a_i,b_i))\,,\;\; m_i \sim \text{uniform}(1,2,3)$$
3. Hierarchical model: individual $q$ for every house, but one prior to sample $q$ from.
$$k_i \sim p(k_i|q_i)\,,\;\; q_i \sim p(q_i|m=(w,a,b))$$

<p align="center">
<img src="hier.png" alt="models" height="200"/>
</p>

#### Exercise 2.2. model selection (5 points)
For models 1 and 3 of Martin's head, derive the formula for the evidence of different priors, implement the formula, and choose the prior with the maximum evidence.

In [ ]:
ev_together = np.zeros((3,))
for i in range(3):
    #YOUR CODE HERE
    
print(ev_together)

In [ ]:
ev_hier = np.ones((3,))
for i in range(3):
    #YOUR CODE HERE
        
print(ev_hier)

#### Exercise 2.3 predictions (2 points)

Under three different models of Martin's head from the previous exercise. For the given house, find the best prior describing the house, find the posterior plot them. Make conclusions in the end.

In [ ]:
def plot_predictions(house,houses_short):
    print("Independent models choice: ",np.array((houses_short.at[house,'ev_1'],houses_short.at[house,'ev_2'],houses_short.at[house,'ev_3'])).argmax()+1)
    print("One model choice: ",ev_together.argmax()+1)
    print("Hierarchical model choice: ",ev_hier.argmax()+1)
    print("ML estimate for q: ",houses_short.at[house,'isAlive']/houses_short.at[house,'People'])
    print("Number of people: ",houses_short.at[house,'People'])

    plt.plot([houses_short.at[house,'isAlive']/houses_short.at[house,'People']],[0],'*',c = 'r',label = 'ML')

    q = np.linspace(0,1,200)
    #separate
    #YOUR CODE HERE
    plt.plot(#YOUR CODE HERE,
             label = 'independent models')
    #together
    #YOUR CODE HERE
    plt.plot(#YOUR CODE HERE,
             label = 'one model')

    #hierarchical
    #YOUR CODE HERE
    plt.plot(#YOUR CODE HERE,
             label = 'hierarchical model')

    plt.legend()
    plt.xlabel('q')
    plt.title('Posterior for 3 different approaches, '+house)
    plt.grid()

In [ ]:
houses_short.index

In [ ]:
house = 'Brave Companions'
plot_predictions(house,houses_short)

In [ ]:
house = 'Brotherhood without banners'
plot_predictions(house,houses_short)

In [ ]:
house = "Night's Watch"
plot_predictions(house,houses_short)

Conclusion:
- Out of three models of Martin's head which models are reasonable, which are not and why?
- How does the number of people in the house affects the posterior?
- Where else you could use this model?